<a href="https://colab.research.google.com/github/quekhyg/NLP-Lyric-Generator/blob/main/%5Bword_level_%2B_whole_corpus%5D_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/SMU_MITB_NLP/NLP project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1MmY0pN1b5xL_C2CijM9ImcFM-UFt4bwr/NLP project


In [2]:
#import packages
import pandas as pd
import numpy as np
import os
import sys 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
tf.random.set_seed(42)

Data Preprocessing

In [4]:
path = "./songs_edit_revised/"
corpus = ''

all_files = os.listdir(path)
for file in all_files:
  with open(os.path.join(path, file)) as f:
    text = f.read()
    corpus+= '\n\n'+ text

In [5]:
components = corpus.split('\n\n')

In [6]:
print(len(components))

288


In [7]:
corpus_list = [string.split('\n') for string in components]
#corpus_list[:10]
for song in corpus_list:
  song.pop(0)

In [8]:
corpus_list[:10]

[[],
 ['Will you make this island',
  'Amazing in all ways',
  'Surprises every corner',
  'Delightful nights and days',
  'Will you take this country',
  'And turn it from a place',
  'To a home that greets you',
  'With smiles on every face',
  'Will you come on this brave journey',
  'Will you help to make it real',
  'Will you write us grand new stories',
  'Songs that everyone will feel'],
 ['So will you swim the current',
  'Will you scale new heights',
  'Will you make it happen',
  'Will you let your dreams take flight',
  'And will you make the difference',
  'Will you seize the day',
  'Will you live each moment',
  'Will you dare to find new ways'],
 ['Will you take this city',
  'And turn it from a place',
  'To a home that greets you',
  'With smiles on every face',
  'Will you come on this brave journey',
  'Will you help to make it real',
  'Will you write us grand new stories',
  'Songs that everyone will feel'],
 ['So will you swim the current',
  'Will you scale new h

In [9]:
df = pd.DataFrame({'song': corpus_list})

df['song'] = df['song'].str.join(', ')
pd.options.display.max_colwidth = 500
df=df.iloc[1:] 
df.head()

,song
1,"Will you make this island, Amazing in all ways, Surprises every corner, Delightful nights and days, Will you take this country, And turn it from a place, To a home that greets you, With smiles on every face, Will you come on this brave journey, Will you help to make it real, Will you write us grand new stories, Songs that everyone will feel"
2,"So will you swim the current, Will you scale new heights, Will you make it happen, Will you let your dreams take flight, And will you make the difference, Will you seize the day, Will you live each moment, Will you dare to find new ways"
3,"Will you take this city, And turn it from a place, To a home that greets you, With smiles on every face, Will you come on this brave journey, Will you help to make it real, Will you write us grand new stories, Songs that everyone will feel"
4,"So will you swim the current, Will you scale new heights, Will you make it happen, Will you let your dreams take flight, And will you make the difference, Will you seize the day, Will you live each moment, Will you dare to find new ways"
5,"Dare to find, Dare to find, Dare to find, New ways"


Generating lyrics for VERSE

In [10]:
#To run this cell if utilities.py is not in the same folder - this will upload utilities.py
# from google.colab import files
# src = list(files.upload().values())[0]
# open('utilities.py','wb').write(src)
# import utilities as U

In [11]:
import utilities as U

In [12]:
"""
Remove punctuations, decontractions, emojis and URLs
"""

df['song'] = df.apply(lambda row : U.remove_punct(row['song']), axis=1)
df['song'] = df.apply(lambda row : U.remove_url(row['song']), axis=1)
df['song'] = df.apply(lambda row : U.remove_emoji(row['song']), axis=1)
df['song'] = df.apply(lambda row : U.decontraction(row['song']), axis=1)
df['song']=  df.apply(lambda row : row['song'].lower(), axis=1)
df.head()

,song
1,will you make this island amazing in all ways surprises every corner delightful nights and days will you take this country and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel
2,so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways
3,will you take this city and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel
4,so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways
5,dare to find dare to find dare to find new ways


In [13]:
songs = df['song'].tolist()

In [14]:
songs[:10]

['will you make this island amazing in all ways surprises every corner delightful nights and days will you take this country and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel',
 'so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways',
 'will you take this city and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel',
 'so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each

In [15]:
# Save verses in .txt file
with open('songs.txt', 'w',encoding="utf-8") as filehandle:  
    for listitem in songs:
        filehandle.write('%s\n' % listitem)

In [16]:
textFileName = 'songs.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()

Word-Level Lyrics Generation

In [17]:
"""
Generate song.
Reference: https://predictivehacks.com/word-based-text-generation/
"""
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
import keras.utils as ku 

tokenizer = Tokenizer()

corpus = raw_text.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
 
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
 
 
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
 
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
 
label = ku.np_utils.to_categorical(label, num_classes=total_words)
 
 
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 97, 100)           107000    
                                                                 
 dropout (Dropout)           (None, 97, 100)           0         
                                                                 
 gru (GRU)                   (None, 128)               88320     
                                                                 
 dense (Dense)               (None, 535)               69015     
                                                                 
 dense_1 (Dense)             (None, 1070)              573520    
                                                                 
Total params: 837,855
Trainable params: 837,855
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
early_stop= tf.keras.callbacks.EarlyStopping(
    monitor="accuracy",
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [19]:
history = model.fit(predictors, label, epochs=50, verbose=1,callbacks = [early_stop])

Epoch 1/50
322/322 [==============================] - 7s 12ms/step - loss: 6.0070 - accuracy: 0.0436
Epoch 2/50
322/322 [==============================] - 3s 9ms/step - loss: 5.2931 - accuracy: 0.0625
Epoch 3/50
322/322 [==============================] - 2s 7ms/step - loss: 4.9688 - accuracy: 0.0865
Epoch 4/50
322/322 [==============================] - 2s 8ms/step - loss: 4.7166 - accuracy: 0.1097
Epoch 5/50
322/322 [==============================] - 2s 8ms/step - loss: 4.4968 - accuracy: 0.1231
Epoch 6/50
322/322 [==============================] - 2s 8ms/step - loss: 4.2909 - accuracy: 0.1455
Epoch 7/50
322/322 [==============================] - 2s 8ms/step - loss: 4.0965 - accuracy: 0.1809
Epoch 8/50
322/322 [==============================] - 2s 7ms/step - loss: 3.9003 - accuracy: 0.2035
Epoch 9/50
322/322 [==============================] - 2s 7ms/step - loss: 3.7231 - accuracy: 0.2402
Epoch 10/50
322/322 [==============================] - 2s 8ms/step - loss: 3.5386 - accuracy: 0.268

In [20]:
'''
Neutral input seeds:
Whenever I think back 
And so this I know 
'''

seed_text = "whenever i think back"
next_words = 100

def generateLyrics(seed_text, model):
  next_words = 100   
  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predicted=np.argmax(model.predict(token_list, verbose=0) ,axis=1)

      output_word = ""
      for word, index in tokenizer.word_index.items():
          if index == predicted:
              output_word = word
              # break
      seed_text += " " + output_word
  return seed_text

lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['whenever i think back i dare to here where we belong well always be the best more more together singapore singapore we can achieve together singapore we can achieve singapore my best and more together singapore singapore together singapore singapore together singapore singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we can achieve together singapore we']

In [21]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("whenever i think back_GRU.txt", header=False, 
                index=False, encoding="utf-8")

from google.colab import files
files.download("whenever i think back_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
seed_text = "and so this I know"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['and so this I know we have when we stand side by nothing as this world where i know that love is dont to strive with the generation new world where we could shine and way to make us till its choices to dream through its your way there out else the world where we thought we have become like long never end lets work together everybody lend a hand cause tomorrows here today dream away come changed lived a future so bright that i call home one drop on the ocean a gem upon the sea where the red and white the moon and']

In [23]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("and so this I know_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("and so this I know_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
'''
Negative input seeds:
I am tired of being what you want me to be 
Feeling so faithless, lost under the surface 
'''

seed_text = "i am tired of being what you want me to be"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['i am tired of being what you want me to be up ive roamed the world its still my home i love my friends i are my friends we are my people my friends we are singapore singaporeans we are singapore we are singapore we will stand together hear the lion roar we are singapore we are singapore were a nation strong and free forevermore this is my country this is my future this is my life this is my family these are my friends we are singapore singaporeans we are singapore we are singapore we will stand together hear the lion roar we are singapore we are singapore were a']

In [25]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("i am tired of being what you want me to be_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("i am tired of being what you want me to be_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
seed_text = "feeling so faithless lost under the surface"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['feeling so faithless lost under the surface whole world will all come on us and i remember what we should be happy cos the best is a million with the dawn drawn everyone by far and proud come for us to be possible to overcome anything theres so much to live in starting a song take flight to will always be the city of the destiny i have a have sown the seeds of hearts and wide youll always be the moment to see this is where we are one big family i want the whole world to know i want to shout it out loud that']

In [27]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("feeling so faithless lost under the surface_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("feeling so faithless lost under the surface_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
'''
Positive input seeds:
Relight our fire, we will find our way 
We will rise stronger together  
'''
seed_text = "relight our fire we will find our way"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['relight our fire we will find our way because if you all grow i care be family i want you breathe i see you smile you have made our lives worthwhile its your warmth your beautiful touch it makes me remember so much of the pleasure i have grown ever memories and been days will you think of who you can help them all that we should be happy because and magic we will start again anew help well make the fire in our hearts touch and heart and feel the spirit make the air stand and bursts to life with you and me that we have you']

In [29]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("relight our fire we will find our way_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("relight our fire we will find our way_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
seed_text = "we will rise stronger together"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['we will rise stronger together by hand in hand for joy and harmony we will strive each day be the best that we can be growing stronger every day well make it you and me well work together hand in hand in joy and harmony we will strive each day be the best that we can be growing stronger every day well make it you and me well work together hand in hand in joy and harmony we will strive each day be the best that we can be growing stronger every day well make it you and me well work together hand in hand']

In [31]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("we will rise stronger together_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("we will rise stronger together_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>